# Hierarchical Demand Reconciliation with Temporal Anomaly Feedback

This notebook demonstrates the hierarchical forecasting system with temporal anomaly feedback on synthetic data.

## Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path().absolute().parent / "src"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Setup complete!")

## Import Project Modules

In [ ]:
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.data.loader import (
    M5DataLoader,
    HierarchicalTimeSeriesDataset,
)
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.data.preprocessing import (
    HierarchicalPreprocessor,
)
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.models.model import (
    HierarchicalReconciliationTransformer,
)
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.evaluation.metrics import (
    HierarchicalMetrics,
)
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.utils.config import (
    load_config,
    set_seed,
)

print("Modules imported successfully!")

## Generate Synthetic Hierarchical Data

In [ ]:
def create_synthetic_hierarchical_data(n_items=30, n_days=180, seed=42):
    """Create synthetic hierarchical time series data."""
    np.random.seed(seed)
    
    # Hierarchy structure
    n_stores = 5
    n_depts = 3
    n_cats = 2
    n_states = 2
    
    # Create items
    items = [f"ITEM_{i:03d}" for i in range(n_items)]
    stores = [f"{['CA', 'TX'][i%n_states]}_{(i%n_stores):01d}" for i in range(n_items)]
    depts = [f"DEPT_{i//10:01d}" for i in range(n_items)]
    cats = [f"CAT_{i//15:01d}" for i in range(n_items)]
    states = [store.split('_')[0] for store in stores]
    
    data = []
    
    for i, (item_id, store_id, dept_id, cat_id, state_id) in enumerate(
        zip(items, stores, depts, cats, states)
    ):
        # Base demand pattern
        base_level = 10 + np.random.uniform(-3, 3)
        trend = np.random.uniform(-0.01, 0.01)
        
        # Weekly seasonality
        weekly_pattern = 3 * np.sin(2 * np.pi * np.arange(n_days) / 7 + np.random.uniform(0, 2*np.pi))
        
        # Random anomalies
        anomaly_days = np.random.choice(n_days, size=np.random.randint(5, 15), replace=False)
        anomaly_effect = np.zeros(n_days)
        anomaly_effect[anomaly_days] = np.random.uniform(5, 20, len(anomaly_days))
        
        # Combine effects
        sales_pattern = (
            base_level
            + trend * np.arange(n_days)
            + weekly_pattern
            + anomaly_effect
            + np.random.normal(0, 1, n_days)
        )
        
        # Ensure non-negative
        sales_pattern = np.maximum(0.1, sales_pattern)
        
        for day in range(n_days):
            data.append({
                "item_id": item_id,
                "dept_id": dept_id,
                "cat_id": cat_id,
                "store_id": store_id,
                "state_id": state_id,
                "d": f"d_{day + 1}",
                "date": pd.Timestamp("2021-01-01") + pd.Timedelta(days=day),
                "sales": sales_pattern[day],
            })
    
    return pd.DataFrame(data)

# Generate data
print("Generating synthetic hierarchical data...")
sales_data = create_synthetic_hierarchical_data()
print(f"Generated {len(sales_data)} records for {sales_data['item_id'].nunique()} items")
print(f"Date range: {sales_data['date'].min()} to {sales_data['date'].max()}")

## Explore the Data

In [ ]:
# Data overview
print("Data Overview:")
print(f"Shape: {sales_data.shape}")
print(f"Columns: {sales_data.columns.tolist()}")
print(f"Unique items: {sales_data['item_id'].nunique()}")
print(f"Unique stores: {sales_data['store_id'].nunique()}")
print(f"Unique departments: {sales_data['dept_id'].nunique()}")
print(f"Date range: {(sales_data['date'].max() - sales_data['date'].min()).days} days")

# Sales statistics
print("\nSales Statistics:")
print(sales_data['sales'].describe())

sales_data.head()

In [ ]:
# Visualize sample time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

# Sample 4 items
sample_items = sales_data['item_id'].unique()[:4]

for i, item in enumerate(sample_items):
    item_data = sales_data[sales_data['item_id'] == item].sort_values('date')
    axes[i].plot(item_data['date'], item_data['sales'], linewidth=1)
    axes[i].set_title(f'Sales for {item}')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Sales')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Hierarchy analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Sales distribution by hierarchy level
store_sales = sales_data.groupby(['store_id', 'date'])['sales'].sum().reset_index()
dept_sales = sales_data.groupby(['dept_id', 'date'])['sales'].sum().reset_index()
total_sales = sales_data.groupby('date')['sales'].sum().reset_index()

# Item level
item_daily_avg = sales_data.groupby('item_id')['sales'].mean()
axes[0].hist(item_daily_avg, bins=20, alpha=0.7, edgecolor='black')
axes[0].set_title('Average Daily Sales Distribution\n(Item Level)')
axes[0].set_xlabel('Average Daily Sales')
axes[0].set_ylabel('Frequency')

# Store level
store_daily_avg = store_sales.groupby('store_id')['sales'].mean()
axes[1].hist(store_daily_avg, bins=10, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_title('Average Daily Sales Distribution\n(Store Level)')
axes[1].set_xlabel('Average Daily Sales')
axes[1].set_ylabel('Frequency')

# Total level time series
axes[2].plot(total_sales['date'], total_sales['sales'], linewidth=2, color='green')
axes[2].set_title('Total Sales Over Time')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Total Sales')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Preprocessing and Dataset Creation

In [ ]:
# Initialize preprocessor
preprocessor = HierarchicalPreprocessor(
    sequence_length=14,  # 2 weeks input
    prediction_length=7, # 1 week prediction
    anomaly_window=7,
    anomaly_threshold=0.1,
)

# Split data
train_data, val_data, test_data = preprocessor.split_data(
    sales_data, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1
)

print(f"Data splits:")
print(f"  Train: {len(train_data)} records")
print(f"  Val: {len(val_data)} records")
print(f"  Test: {len(test_data)} records")

# Fit preprocessor on training data
preprocessor.fit(train_data)

# Transform data
train_processed = preprocessor.transform(train_data)
val_processed = preprocessor.transform(val_data)

print(f"\nProcessed data columns: {train_processed.columns.tolist()}")
print(f"Anomalies detected in training: {train_processed['is_anomaly'].sum()} / {len(train_processed)}")

In [ ]:
# Visualize anomaly detection
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Sample item for visualization
sample_item = train_processed['item_id'].iloc[0]
item_data = train_processed[train_processed['item_id'] == sample_item].sort_values('date')

# Original sales with anomalies highlighted
axes[0].plot(item_data['date'], item_data['sales'], 'b-', linewidth=1, label='Sales')
anomaly_points = item_data[item_data['is_anomaly'] == True]
if len(anomaly_points) > 0:
    axes[0].scatter(anomaly_points['date'], anomaly_points['sales'], 
                   color='red', s=50, label='Detected Anomalies', zorder=5)
axes[0].set_title(f'Sales Time Series with Anomaly Detection: {sample_item}')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Sales')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

# Anomaly scores
axes[1].plot(item_data['date'], item_data['anomaly_score'], 'g-', linewidth=1, label='Anomaly Score')
axes[1].axhline(y=0.5, color='red', linestyle='--', label='Threshold')
axes[1].set_title('Anomaly Scores Over Time')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Anomaly Score')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create datasets
hierarchy_mapping = {
    "item_id": 0,
    "dept_id": 1,
    "cat_id": 2,
    "store_id": 3,
    "state_id": 4,
    "total": 5,
}

train_dataset = HierarchicalTimeSeriesDataset(
    data=train_processed,
    sequence_length=14,
    prediction_length=7,
    hierarchy_mapping=hierarchy_mapping,
    features=["sales"],
)

val_dataset = HierarchicalTimeSeriesDataset(
    data=val_processed,
    sequence_length=14,
    prediction_length=7,
    hierarchy_mapping=hierarchy_mapping,
    features=["sales"],
)

print(f"Dataset sizes:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val: {len(val_dataset)} samples")

# Examine a sample
sample = train_dataset[0]
print(f"\nSample structure:")
for key, value in sample.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape {value.shape}, dtype {value.dtype}")
    else:
        print(f"  {key}: {value}")

## Model Initialization and Architecture

In [ ]:
# Create a simple reconciliation matrix for demonstration
n_levels = len(hierarchy_mapping)
reconciliation_matrix = torch.eye(n_levels, dtype=torch.float32)

# Initialize model
model = HierarchicalReconciliationTransformer(
    input_dim=1,
    hidden_size=64,  # Smaller for demo
    num_layers=2,
    num_heads=4,
    dropout=0.1,
    prediction_length=7,
    num_hierarchy_levels=n_levels,
    reconciliation_matrix=reconciliation_matrix,
    learning_rate=1e-3,
    weight_decay=1e-4,
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model initialized:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB")

In [ ]:
# Test model forward pass
model.eval()

# Create a small batch
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)
batch = next(iter(train_loader))

print(f"Batch shapes:")
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")

# Forward pass
with torch.no_grad():
    outputs = model(batch["input"], batch["hierarchy_level"])

print(f"\nModel outputs:")
for key, value in outputs.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape {value.shape}, range [{value.min():.3f}, {value.max():.3f}]")
    else:
        print(f"  {key}: {value}")

## Model Training (Mini Demo)

In [ ]:
# Quick training demo (just a few steps for illustration)
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Set model to training mode
model.train()

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# Training loop (just a few batches for demo)
train_losses = []
n_demo_batches = 10

print("Running mini training demo...")

for batch_idx, batch in enumerate(train_loader):
    if batch_idx >= n_demo_batches:
        break
        
    # Forward pass
    outputs = model(batch["input"], batch["hierarchy_level"])
    
    # Compute loss (simplified)
    forecast_loss = F.mse_loss(outputs["base_forecasts"], batch["target"])
    reconciliation_loss = F.mse_loss(outputs["reconciled_forecasts"], batch["target"])
    coherence_loss = outputs["coherence_loss"]
    
    total_loss = forecast_loss + 0.3 * reconciliation_loss + 0.1 * coherence_loss
    
    # Backward pass
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    train_losses.append(total_loss.item())
    
    if batch_idx % 3 == 0:
        print(f"  Batch {batch_idx}: Loss = {total_loss.item():.4f}")

print(f"\nDemo training completed. Final loss: {train_losses[-1]:.4f}")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, 'b-', linewidth=2, marker='o')
plt.title('Training Loss During Mini Demo')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

## Model Evaluation and Predictions

In [ ]:
# Evaluate model on validation set
model.eval()

all_predictions = []
all_targets = []
all_base_predictions = []
all_reconciled_predictions = []
all_anomaly_scores = []
all_hierarchy_levels = []

with torch.no_grad():
    for batch in val_loader:
        outputs = model(batch["input"], batch["hierarchy_level"])
        
        all_predictions.append(outputs["reconciled_forecasts"].numpy())
        all_targets.append(batch["target"].numpy())
        all_base_predictions.append(outputs["base_forecasts"].numpy())
        all_reconciled_predictions.append(outputs["reconciled_forecasts"].numpy())
        all_anomaly_scores.append(outputs["anomaly_scores"].numpy())
        all_hierarchy_levels.append(batch["hierarchy_level"].numpy())

# Concatenate results
predictions = np.concatenate(all_predictions, axis=0)
targets = np.concatenate(all_targets, axis=0)
base_predictions = np.concatenate(all_base_predictions, axis=0)
reconciled_predictions = np.concatenate(all_reconciled_predictions, axis=0)
anomaly_scores = np.concatenate(all_anomaly_scores, axis=0)
hierarchy_levels = np.concatenate(all_hierarchy_levels, axis=0)

print(f"Evaluation completed:")
print(f"  Predictions shape: {predictions.shape}")
print(f"  Targets shape: {targets.shape}")
print(f"  Number of samples: {len(predictions)}")

In [ ]:
# Compute basic metrics
from hierarchical_demand_reconciliation_with_temporal_anomaly_feedback.evaluation.metrics import (
    HierarchicalMetrics
)

# Flatten predictions for metric computation
predictions_flat = predictions.reshape(-1)
targets_flat = targets.reshape(-1)
base_predictions_flat = base_predictions.reshape(-1)
reconciled_predictions_flat = reconciled_predictions.reshape(-1)
hierarchy_levels_flat = np.repeat(hierarchy_levels, predictions.shape[1])

# Initialize metrics
hierarchical_metrics = HierarchicalMetrics()

# Compute metrics
metrics = hierarchical_metrics.compute_all_metrics(
    predictions=reconciled_predictions_flat,
    targets=targets_flat,
    hierarchy_levels=hierarchy_levels_flat,
    anomaly_scores=anomaly_scores.flatten(),
)

# Display key metrics
print("\n" + "="*50)
print("EVALUATION METRICS")
print("="*50)

key_metrics = ['rmse', 'mae', 'mape', 'smape', 'wrmsse']
for metric in key_metrics:
    if metric in metrics:
        print(f"{metric.upper()}: {metrics[metric]:.4f}")

print(f"\nAnomaly Detection Rate: {metrics.get('anomaly_detection_rate', 'N/A'):.3f}")
print(f"Samples Evaluated: {metrics.get('n_samples', 'N/A')}")
print("="*50)

## Visualization of Results

In [ ]:
# Prediction vs target scatter plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Overall predictions vs targets
axes[0].scatter(targets_flat, reconciled_predictions_flat, alpha=0.5, s=1)
min_val = min(targets_flat.min(), reconciled_predictions_flat.min())
max_val = max(targets_flat.max(), reconciled_predictions_flat.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect prediction')
axes[0].set_xlabel('True Values')
axes[0].set_ylabel('Predictions')
axes[0].set_title('Reconciled Predictions vs Targets')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Base vs reconciled predictions
axes[1].scatter(base_predictions_flat, reconciled_predictions_flat, alpha=0.5, s=1, color='green')
min_val = min(base_predictions_flat.min(), reconciled_predictions_flat.min())
max_val = max(base_predictions_flat.max(), reconciled_predictions_flat.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', label='No change')
axes[1].set_xlabel('Base Predictions')
axes[1].set_ylabel('Reconciled Predictions')
axes[1].set_title('Base vs Reconciled Predictions')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Anomaly score distribution
axes[2].hist(anomaly_scores.flatten(), bins=30, alpha=0.7, edgecolor='black', color='orange')
axes[2].axvline(0.5, color='red', linestyle='--', label='Threshold')
axes[2].set_xlabel('Anomaly Score')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Anomaly Score Distribution')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Performance by hierarchy level
unique_levels = np.unique(hierarchy_levels)
level_names = [f"Level {level}" for level in unique_levels]
level_errors = []
level_counts = []

for level in unique_levels:
    level_mask = hierarchy_levels == level
    if np.any(level_mask):
        level_preds = reconciled_predictions[level_mask].flatten()
        level_targets = targets[level_mask].flatten()
        level_error = np.mean(np.abs(level_preds - level_targets))
        level_errors.append(level_error)
        level_counts.append(np.sum(level_mask))

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MAE by hierarchy level
bars1 = axes[0].bar(level_names, level_errors, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Hierarchy Level')
axes[0].set_ylabel('Mean Absolute Error')
axes[0].set_title('Performance by Hierarchy Level')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Add value labels on bars
for bar, error in zip(bars1, level_errors):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{error:.3f}', ha='center', va='bottom')

# Sample counts by hierarchy level
bars2 = axes[1].bar(level_names, level_counts, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('Hierarchy Level')
axes[1].set_ylabel('Number of Samples')
axes[1].set_title('Sample Distribution by Hierarchy Level')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars2, level_counts):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{count}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Sample prediction visualization
# Show actual vs predicted time series for a few samples

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

# Select first 4 samples
for i in range(min(4, len(predictions))):
    sample_target = targets[i, :, 0]  # Shape: (prediction_length,)
    sample_base_pred = base_predictions[i, :, 0]
    sample_reconciled_pred = reconciled_predictions[i, :, 0]
    
    time_steps = range(len(sample_target))
    
    axes[i].plot(time_steps, sample_target, 'b-', linewidth=2, label='Target', marker='o')
    axes[i].plot(time_steps, sample_base_pred, 'r--', linewidth=2, label='Base Prediction', marker='s')
    axes[i].plot(time_steps, sample_reconciled_pred, 'g-', linewidth=2, label='Reconciled Prediction', marker='^')
    
    axes[i].set_title(f'Sample {i+1} (Level {hierarchy_levels[i]})')
    axes[i].set_xlabel('Time Step')
    axes[i].set_ylabel('Sales')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary and Conclusions

In [ ]:
print("\n" + "="*60)
print("HIERARCHICAL DEMAND RECONCILIATION DEMO SUMMARY")
print("="*60)

print("\n📊 Data:")
print(f"  • Generated {sales_data['item_id'].nunique()} synthetic items")
print(f"  • {len(sales_data)} total observations")
print(f"  • {sales_data['date'].nunique()} time periods")
print(f"  • Hierarchy: {len(hierarchy_mapping)} levels")

print("\n🤖 Model:")
print(f"  • Transformer-based hierarchical forecasting")
print(f"  • {trainable_params:,} trainable parameters")
print(f"  • Anomaly detection with attention mechanism")
print(f"  • Reconciliation with coherence constraints")

print("\n📈 Performance:")
for metric in ['rmse', 'mae', 'mape', 'wrmsse']:
    if metric in metrics:
        print(f"  • {metric.upper()}: {metrics[metric]:.4f}")

print(f"  • Anomaly detection rate: {metrics.get('anomaly_detection_rate', 0):.1%}")
print(f"  • Evaluated on {metrics.get('n_samples', 0)} samples")

print("\n🎯 Key Features Demonstrated:")
print("  ✓ Hierarchical time series forecasting")
print("  ✓ Temporal anomaly detection")
print("  ✓ Forecasting reconciliation")
print("  ✓ Multi-level hierarchy handling")
print("  ✓ Attention-based temporal modeling")

print("\n📝 Next Steps:")
print("  • Run full training: `python scripts/train.py --synthetic-data`")
print("  • Evaluate on M5 data: `python scripts/train.py --data-path data/m5`")
print("  • Tune hyperparameters in configs/default.yaml")
print("  • Analyze reconciliation improvements")

print("\n" + "="*60)
print("Demo completed successfully! 🎉")
print("="*60)